In [0]:
#!apt-get install protobuf-compiler python-pil python-lxml python-tk
#!pip install Cython
#!git clone https://github.com/tensorflow/models.git
#!cd models/research; protoc object_detection/protos/*.proto --python_out=.
#!cd models/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim:; python object_detection/builders/model_builder_test.py
#!git clone https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10.git
import sys, os
sys.path.append('/models/research')
sys.path.append('/models/research/slim')
sys.path.append('/models/research/object_detection')
os.environ['PYTHONPATH'] += ":/models:/models/research:/models/research/slim/"
os.environ['PYTHONPATH'] += ":/content/models:/content/models/research:/content/models/research/slim/"

In [0]:
#!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
#!tar -xvzf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
#!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz
#!tar -xvzf ssd_mobilenet_v1_coco_2018_01_28.tar.gz

In [0]:
#!rm TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/train/*
#!rm TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/test/*
#!rm TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/test_labels.csv
#!rm TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/train_labels.csv
#!rm TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/training/*
#!rm TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/inference_graph/*

In [0]:
#/content/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/train
#/content/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/test
#!unzip train.zip -d ./TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/train/
#!unzip test.zip -d ./TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/test/
#!cd TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/;python xml_to_csv.py

In [0]:
## EDIT generate_tfrecord.py first 
#!mv generate_tfrecord.py TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/

In [0]:
!mv TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/* ./
!cd models/research/; python setup.py build
!cd models/research/; python setup.py install
#!python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record
#!python generate_tfrecord.py --csv_input=images/test_labels.csv --image_dir=images/test --output_path=test.record

In [0]:
#!mkdir models/research/object_detection/training
#!mv labelmap.pbtxt models/research/object_detection/training/

In [0]:
#!cp models/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config ./training/

In [0]:

#!cd faster_rcnn_inception_v2_coco_2018_01_28/;pwd
#!mv faster_rcnn_inception_v2_pets.config ./training/
#!cp models/research/object_detection/legacy/train.py models/research/object_detection/
#!cp faster_rcnn_inception_v2_pets.config models/research/object_detection/training/
!cd models/research/object_detection/; python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/faster_rcnn_inception_v2_pets.config

In [0]:
!cd models/research/object_detection/;python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix training/model.ckpt-11177 --output_directory inference_graph

In [0]:
#!echo $PYTHONPATH
#!rm -rf models/research/object_detection/training/
#!mkdir models/research/object_detection/training
#!cp labelmap.pbtxt models/research/object_detection/training/
!cp faster_rcnn_inception_v2_pets.config models/research/object_detection/training/
#!cp ssd_mobilenet_v1_coco.config models/research/object_detection/training/
#!cd models/research/slim/nets/
#!cd models/;export PYTHONPATH=$PYTHONPATH:pwd:pwd/slim

In [0]:
!cp -r models/research/object_detection/inference_graph ./

In [0]:
#!rm -rf inference_graph/*

In [0]:
!zip -r faster-rcnn_11177_epoch_1_nonmax_suppress.zip inference_graph/*

In [20]:
import tensorflow as tf
import numpy as np
import cv2

class ObjectDetect():

    counter = 0
    def __init__(self):
        self.g2 = tf.Graph()
        with self.g2.as_default():
            self.sess = tf.Session(graph=self.g2)
            with self.sess.as_default():
                saver = tf.train.import_meta_graph('inference_graph/model.ckpt.meta')
                saver.restore(self.sess, "inference_graph/model.ckpt")

    def pedd(self,img):
      img_list = [img]
      with self.g2.as_default(),self.sess.as_default():
          ip = tf.get_default_graph().get_tensor_by_name("image_tensor:0")
          boxes = tf.get_default_graph().get_tensor_by_name("detection_boxes:0")
          scores = tf.get_default_graph().get_tensor_by_name("detection_scores:0")
          classes = tf.get_default_graph().get_tensor_by_name("detection_classes:0")
          b,s,c  = self.sess.run([boxes,scores,classes], feed_dict={ip: img_list})

      b = np.squeeze(b)
      bbs = []
      print(c)
      b = [b]
      for i in range(len(b)):
            #if((b[i][0]>0 or b[i][0]>0 or b[i][0]>0 or b[i][0]>0) and c[0][i]==1 and s[0][i]>0.4):
            
            if ((b[i][0] > 0 or b[i][0] > 0 or b[i][0] > 0 or b[i][0] > 0)):
              y1,x1,y2,x2 = int(b[i][0]*img.shape[0]), int(b[i][1]*img.shape[1]), int(b[i][2]*img.shape[0]), int(b[i][3]*img.shape[1])
              #cv2.imwrite('videos/'+str(self.counter)+str(i)+'.jpg',img[y1:y2, x1:x2])
              #self.counter += 1

              bbs.append([x1,y1,x2,y2])

      new_bbs = bbs
      for i in new_bbs:
          x1,y1,x2,y2 = i[0],i[1],i[2],i[3]
          cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
      return img,new_bbs


od = ObjectDetect()
  

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from inference_graph/model.ckpt


In [0]:
#!unzip test.zip

In [22]:
frame = cv2.imread('negative_image.png')
frame,bbs = od.pedd(frame)
cv2.imwrite('out.jpg',frame)

[[1.]]


True